# Stixrude Pure Phases:  SymPy Code Generation

This notebook generates a model for a pure phase (single endmember) for all Mg2SiO4 and Fe2SiO4 compositions in Stixrude

* Forsterite
* MgWadsleyite
* MgRingwoodite
* Fayalite
* FeWadsleyite
* FeRingwoodite

In [ ]:
import os,sys
import pandas as pd
import numpy as np
import sympy as sym
import hashlib
import time
sym.init_printing()

Required ENKI packages

In [ ]:
from thermocodegen.coder import coder

### let's set up some directory names for clarity

In [ ]:
HOME_DIR = os.path.abspath(os.curdir)
SPUD_DIR = HOME_DIR+'/../phases'

try:
    os.mkdir(SPUD_DIR)
except:
    pass

Set a reference string for this Notebook

In [ ]:
reference = 'Thermocodegen-v0.6/share/thermocodegen/examples/Systems/MgFeSiO4_stixrude/notebooks/Generate_pure_phases.ipynb'

## Pure Phase


### Number of solution components
This notebook illustrates a pure phase 

In [ ]:
c = 1

### Create a pure phase using the SimpleSolution Model (with only one endmember)

In [ ]:
model = coder.SimpleSolnModel.from_type(nc=c)

### Retrieve primary compositional variables
- $n$ is a vector of mole numbers of each component  
- $n_T$ is the total number of moles in the solution

### and construct a derived mole fraction variable
- $X$ is a vector of mole fractions of components in the system

In [ ]:
n = model.n
nT = model.nT
X = n/nT
n, nT, X

### Retrieve the temperature, pressure, and standard state chemical potentials
- $T$ is temperature in $K$
- $P$ is pressure in $bars$
- $\mu$ in Joules

In [ ]:
T = model.get_symbol_for_t()
P = model.get_symbol_for_p()
mu = model.mu
T,P,mu

Check model dictionary

In [ ]:
model.model_dict

### Define the standard state contribution to solution properties

In [ ]:
G_ss = (n.transpose()*mu)[0]
G_ss

### Define the Gibbs free energy of solution

In [ ]:
G = G_ss  
G

### Add the Gibbs free energy of solution to the model

In [ ]:
model.add_potential_to_model('G',G)

### let's inspect the dictionary and unset parameters

In [ ]:
model.model_dict

In [ ]:
values_dict = model.get_values()
values_dict

## Sort out Formula conversion strings

* assign a formula string for code generation  
    * here the solution formula is Mg$_x$Fe$_{2-x}$SiO$_4$ where $x\in[0,2]$
* assign a conversion string to map element concentrations to moles of end members
    * here endmember 0 is Forsterite, ${\rm{Mg_2SiO_4}}$ so 1 mole of Forsterite will be 0.5* the number of Mg atoms
    * likewise for endmember 1, Fayalite,  1 mole of Fa will be 0.5* the number of Fe atoms whic gives the conversion_string for moles of endmember 0, and 1 as 
    
    `conversion_string=['[0]=0.5*[Mg]', '[1]=0.5*[Fe]']`


In [ ]:
values_dict.update(dict(formula_string='Mg[Mg]SiO4',
                        conversion_string=['[0]=0.5*[Mg]'],
                        test_string = ['[0] > 0.0'],
                        T_r = 300.0))
values_dict

### Add additional parameters for Pure Mg2SiO4 and Fe2SiO4 phases

In [ ]:
Fo_dict = dict(name='Forsterite',abbrev='Fo',
                        reference=reference,
                        endmembers = ['Forsterite_stixrude'])
MgWa_dict = dict(name='MgWadsleyite',abbrev='MgWa',
                        reference=reference,
                        endmembers = ['MgWadsleyite_stixrude'])
MgRi_dict = dict(name='MgRingwoodite',abbrev='MgRi',
                        reference=reference,
                        endmembers = ['MgRingwoodite_stixrude'])
Fa_dict = dict(name='Fayalite',abbrev='Fa',
                        reference=reference,
                        endmembers = ['Fayalite_stixrude'])
FeWa_dict = dict(name='FeWadsleyite',abbrev='FeWa',
                        reference=reference,
                        endmembers = ['FeWadsleyite_stixrude'])
FeRi_dict = dict(name='FeRingwoodite',abbrev='FeRi',
                        reference=reference,
                        endmembers = ['FeRingwoodite_stixrude'])

dict_list = [ Fo_dict, MgWa_dict, MgRi_dict, Fa_dict, FeWa_dict, FeRi_dict ]

### Add additional parameters for solid phases Perovskites and oxides

In [ ]:
MgPv_dict = dict(name='MgPerovskite',abbrev='MgPv',
                        reference=reference,
                        endmembers = ['MgPerovskite_stixrude'],
                        formula_string='Mg[Mg]SiO3',
                        conversion_string=['[0]=[Mg]'])
Pe_dict = dict(name='Periclase',abbrev='Pe',
                        reference=reference,
                        endmembers = ['Periclase_stixrude'],
                        formula_string='Mg[Mg]O',
                        conversion_string=['[0]=[Mg]'])
FePv_dict = dict(name='FePerovskite',abbrev='FePv',
                        reference=reference,
                        endmembers = ['FePerovskite_stixrude'],
                        formula_string='Fe[Fe]SiO3',
                        conversion_string=['[0]=[Fe]'])
Wu_dict = dict(name='Wuestite',abbrev='Wu',
                        reference=reference,
                        endmembers = ['Wuestite_stixrude'],
                        formula_string='Fe[Fe]O',
                        conversion_string=['[0]=[Fe]'])
dict_list += [ MgPv_dict, Pe_dict, FePv_dict, Wu_dict]

## Generate Spud XML files

### dump spudfiles

In [ ]:
for d in dict_list:
    values_dict.update(d)
    model.set_values(values_dict)
    file = model.to_xml(path=SPUD_DIR)
    print(file)